## Assignment 11
## Kimberly Cable

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.chdir('/content/drive/My Drive/DSC650/assignment11')
!pwd

Mounted at /content/drive
/content/drive/My Drive/DSC650/assignment11


In [2]:
import keras
import numpy as np

Download and parse writings of Nietzche

In [3]:
path = 'House_On_The_Cliff.txt'

text = open(path).read().lower()

print('Corpus length', len(text))

Corpus length 246829


In [4]:
np.random.seed(seed = 15)

Vectorize sequences of characters

In [5]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print(f'Number of sequences: {len(sentences)}')
print(f'Sample: {sentences[10]}')

Number of sequences: 82257
Sample: f the house on the cliff, by franklin w.
dixon

this ebook i


In [6]:
chars = sorted(list(set(text)))

print(f'Unique characters: {len(chars)}')

Unique characters: 64


In [7]:
char_indices = dict((char, chars.index(char)) for char in chars) 

print('Vectorization...')

Vectorization...


In [8]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  
  y[i, char_indices[next_chars[i]]] = 1

Build single-layer LSTM model for next-character prediction

In [9]:
from keras import layers

In [10]:
model = keras.models.Sequential()

model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Model compilation configuration

In [11]:
import tensorflow as tf

optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Function to sample the next character given the model's predictions

In [12]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  
  return np.argmax(probas)

Fit the Model

In [15]:
import random
import sys

for epoch in range(1, 60):
  print(f'\n====== EPOCH: {epoch} ======')

  model.fit(x, y, batch_size=128, epochs=1)


====== EPOCH: 1 ======
643/643 [==============================] - 126s 195ms/step - loss: 1.0612

====== EPOCH: 2 ======
643/643 [==============================] - 125s 194ms/step - loss: 1.0519

====== EPOCH: 3 ======
643/643 [==============================] - 126s 197ms/step - loss: 1.0509

====== EPOCH: 4 ======
643/643 [==============================] - 124s 194ms/step - loss: 1.0415

====== EPOCH: 5 ======
643/643 [==============================] - 126s 197ms/step - loss: 1.0387

====== EPOCH: 6 ======
643/643 [==============================] - 128s 199ms/step - loss: 1.0360

====== EPOCH: 7 ======
643/643 [==============================] - 127s 198ms/step - loss: 1.0286

====== EPOCH: 8 ======
643/643 [==============================] - 126s 197ms/step - loss: 1.0249

====== EPOCH: 9 ======
643/643 [==============================] - 127s 197ms/step - loss: 1.0179

====== EPOCH: 10 ======
643/643 [==============================] - 128s 199ms/step - loss: 1.0119

====== EPOCH: 11 =

Generate Text

In [16]:
'''
# Selects a text seed at random:
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]

print(f'--- Generating with seed: "{generated_text}"')

for temperature in [0.2, 0.5, 1.0, 1.2]: 
  print(f'\n------ temperature: {temperature}\n')

  sys.stdout.write(generated_text)
  
  for i in range(400):
    # One-hot encodes characters generated so far:
    sampled = np.zeros((1, maxlen, len(chars)))

    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
    
    # Samples the next character
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    
    sys.stdout.write(next_char)
  
  print()
  '''

'\n# Selects a text seed at random:\nstart_index = random.randint(0, len(text) - maxlen - 1)\ngenerated_text = text[start_index: start_index + maxlen]\n\nprint(f\'--- Generating with seed: "{generated_text}"\')\n\nfor temperature in [0.2, 0.5, 1.0, 1.2]: \n  print(f\'\n------ temperature: {temperature}\n\')\n\n  sys.stdout.write(generated_text)\n  \n  for i in range(400):\n    # One-hot encodes characters generated so far:\n    sampled = np.zeros((1, maxlen, len(chars)))\n\n    for t, char in enumerate(generated_text):\n      sampled[0, t, char_indices[char]] = 1.\n    \n    # Samples the next character\n    preds = model.predict(sampled, verbose=0)[0]\n    next_index = sample(preds, temperature)\n    next_char = chars[next_index]\n    generated_text += next_char\n    generated_text = generated_text[1:]\n    \n    sys.stdout.write(next_char)\n  \n  print()\n  '

Generate 20 samples

In [17]:
for i in range(0, 20):
  print(f'\n====== GENERATED TEXT #{i+1}:\n')
 
  # Selects a text seed at random:
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  
  print('--- Generating with seed: "' + generated_text + '"')
  
  for temperature in [0.2, 0.5, 1.0, 1.2]: # Tries a range of different sampling temperatures
    print('\n------ temperature:', temperature, '\n')

    sys.stdout.write(generated_text)
    
    for i in range(400): # Generates 400 characters, starting from seed text
      # One-hot encodes characters generated so far:
      sampled = np.zeros((1, maxlen, len(chars)))
      
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
      
      # Samples the next character
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)
    
    print()


====== GENERATED TEXT #1:

--- Generating with seed: "ly struggling for a few minutes he was able to
work his way "

------ temperature: 0.2 

ly struggling for a few minutes he was able to
work his way any one road to the cove and the man who was a steep of the cliff.

frank and
there was a contine that they were and the farmer and the farmhouse.

they went out to the state when the hardy boys were to loos any project gutes and the hardy boys, but the hardy boys, but what do the smugglers were and the man who was a stranger.

"what do you shoot and he'll better a smugglers!"

an he was a cover o

------ temperature: 0.5 

ou shoot and he'll better a smugglers!"

an he was a cover on the
door.

"i guess that it would be man han had a smuggler the back and
they were standing in the cove and the farmhouse.

they went on sceepting the bay.

the wetken
they left the walk. the words had been each right a close to be his
brought had sure the farmhouse and still remained a door who was a sud

<ipython-input-12-c74a7afafd2e>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


g™
must biff the sight, hose.

smugglers they motion close to shoot or scann cock use of scepter, and the door was not himself loos up
to the startling that
that had come walker
that his face. there were
started hkan we has and keep immediately.

------ temperature: 1.2 

s face. there were
started hkan we has and keep immediately. no casion went divknations, but
he came along the yust along the rowboat."

noise, distremed out from he was hidden assitionaled creeved extenture

in the batt., if you didn't tew the hardy gotsn'e, lookin'tly. "it  play decrappr look expla out of him. placaged to chance of detective st dismid around them
if you time in the other too "in the move of smuggle again, they went speep case your im
tha

====== GENERATED TEXT #9:

--- Generating with seed: ""and it wasn't any fun hiding in that attic with the bullets"

------ temperature: 0.2 

"and it wasn't any fun hiding in that attic with the bullets and
the hardy boys were out of the house with the other agaed